# Library

In [ ]:
pip install comet_ml

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import warnings
import joblib            as jlb
warnings.filterwarnings('ignore')
from sklearn.decomposition   import PCA
from sklearn.preprocessing   import LabelEncoder, OneHotEncoder
from sklearn.preprocessing   import Binarizer, PolynomialFeatures, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors       import KNeighborsClassifier
from comet_ml                import ConfusionMatrix
from sklearn                 import metrics
from sklearn.metrics         import accuracy_score, confusion_matrix
from sklearn.metrics         import auc,roc_curve
from sklearn.metrics         import precision_score, recall_score, f1_score
from sklearn.metrics         import roc_auc_score
from sklearn.model_selection import cross_val_score

# Preprocessing

##Import dataset##

In [ ]:
!gdown 1DZRQRJNDcnK79EU4Uhlp3TILiVM9-d9T

Downloading...
From: https://drive.google.com/uc?id=1DZRQRJNDcnK79EU4Uhlp3TILiVM9-d9T
To: /content/House_data.csv
100% 444k/444k [00:00<00:00, 67.9MB/s]


In [ ]:
df = pd.read_csv('House_data.csv')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [ ]:
df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SalePrice,Condition
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,208500,NOR
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,181500,NOR
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,223500,NOR
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,140000,ABN
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,250000,NOR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,175000,NOR
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,210000,NOR
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,266500,NOR
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,142125,NOR


##Missing values##

In [ ]:
null_counts = df.isnull().sum()
print(null_counts[null_counts > 0])

LotFrontage      259
Alley           1369
MasVnrType         8
MasVnrArea         8
BsmtQual          37
BsmtCond          37
BsmtExposure      38
BsmtFinType1      37
BsmtFinType2      38
Electrical         1
FireplaceQu      690
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
GarageCond        81
PoolQC          1453
Fence           1179
MiscFeature     1406
dtype: int64


In [ ]:
df = df.drop(columns=['Alley', 'PoolQC', 'Fence', 'MiscFeature'])

In [ ]:
df['FireplaceQu'] = df['FireplaceQu'].fillna("N/A")
df['Electrical'] = df['Electrical'].fillna("N/A")

In [ ]:
garage_columns = df.filter(like='Garage')
for column in garage_columns.columns:
  if column != 'GarageYrBlt':
    df[column].fillna("N/A", inplace=True)
  else:
    mode_value = df[column].mode()[0]
    df[column].fillna(mode_value, inplace=True)

In [ ]:
bsmt_columns = df.filter(like='Bsmt')
df[bsmt_columns.columns] = bsmt_columns.fillna("N/A")

In [ ]:
masvnr_columns = df.filter(like='MasVnr')
for column in masvnr_columns.columns:
    mode_value = df[column].mode()[0]
    df[column].fillna(mode_value, inplace=True)

In [ ]:
df['LotFrontage_new'] = df.LotFrontage.interpolate()
df.drop(columns = 'LotFrontage', inplace = True)

##Noisy data##

In [ ]:
def basigma(col, df):
  lech3sigma_tren = round(df[col].mean() + 3 * df[col].std(ddof = 1))
  lech3sigma_duoi = round(df[col].mean() - 3 * df[col].std(ddof = 1))
  df_filter = df[(df[col] > lech3sigma_tren) | (df[col] < lech3sigma_duoi)]
  if df_filter[col].count() > 0:
    print(f'Cột {i} có {df_filter[col].count()} giá trị nhiễu là các giá trị ngoài khoảng ({lech3sigma_duoi}, {lech3sigma_tren})')
    # print(sorted(df_filter[col].unique()))

  # thay thế nhiễu bằng trung vị
  trungvi = df[col].median()
  df.loc[(df[col] > lech3sigma_tren) | (df[col] < lech3sigma_duoi), col] = trungvi

In [ ]:
print('Thay thế nhiễu bằng trung vị ở các cột sau')
for i in df.select_dtypes(include = 'number').columns:
  basigma(i, df)

Thay thế nhiễu bằng trung vị ở các cột sau
Cột MSSubClass có 30 giá trị nhiễu là các giá trị ngoài khoảng (-70, 184)
Cột LotArea có 13 giá trị nhiễu là các giá trị ngoài khoảng (-19427, 40461)
Cột OverallQual có 2 giá trị nhiễu là các giá trị ngoài khoảng (2, 10)
Cột OverallCond có 1 giá trị nhiễu là các giá trị ngoài khoảng (2, 9)
Cột YearBuilt có 6 giá trị nhiễu là các giá trị ngoài khoảng (1881, 2062)
Cột MasVnrArea có 32 giá trị nhiễu là các giá trị ngoài khoảng (-439, 645)
Cột BsmtFinSF1 có 6 giá trị nhiễu là các giá trị ngoài khoảng (-925, 1812)
Cột BsmtFinSF2 có 49 giá trị nhiễu là các giá trị ngoài khoảng (-437, 531)
Cột BsmtUnfSF có 11 giá trị nhiễu là các giá trị ngoài khoảng (-758, 1893)
Cột TotalBsmtSF có 10 giá trị nhiễu là các giá trị ngoài khoảng (-259, 2374)
Cột 1stFlrSF có 12 giá trị nhiễu là các giá trị ngoài khoảng (3, 2322)
Cột 2ndFlrSF có 4 giá trị nhiễu là các giá trị ngoài khoảng (-963, 1657)
Cột LowQualFinSF có 20 giá trị nhiễu là các giá trị ngoài khoảng (-140,

##Selection##

In [ ]:
df = df.drop(columns = 'Id')

**Loại bỏ các cột phân loại có độ biến thiên yếu**

In [ ]:
def check(col, df):
  group = df.groupby(col)['LotArea'].count()
  group = pd.DataFrame(group.reset_index())
  dieukien = (group['LotArea'].sum() - group['LotArea'].max())  / group['LotArea'].max()
  if dieukien < 0.2:
    df.drop(columns = col, inplace = True)

In [ ]:
for i in df.select_dtypes(include = 'object').columns:
  if i != 'Condition':
    check(i,df)

##Transform##

In [ ]:
def onehot(col, df):
  if col != 'Condition':
    enc = OneHotEncoder(sparse = False) # dạng
    df_enc = pd.DataFrame(enc.fit_transform(df[[col]]), columns = df[col].unique())
    df.drop(columns = col, inplace = True)
    df_combined = pd.concat([df, df_enc], axis=1)
    return df_combined

In [ ]:
def label(col, df):
  if col != 'Condition':
    enc = LabelEncoder()
    df[str(col)+'_new']  = pd.Series(enc.fit_transform(df[col]))
    df.drop(columns = col, inplace = True)
    mappings = {index: label for index, label in enumerate(enc.classes_)}
    print(mappings)

In [ ]:
for i in df.select_dtypes(include = ['object']).columns:
  if len(df[i].unique()) < 6:
    df = onehot(i, df)
  else:
    label(i,df)

{0: 'Blmngtn', 1: 'Blueste', 2: 'BrDale', 3: 'BrkSide', 4: 'ClearCr', 5: 'CollgCr', 6: 'Crawfor', 7: 'Edwards', 8: 'Gilbert', 9: 'IDOTRR', 10: 'MeadowV', 11: 'Mitchel', 12: 'NAmes', 13: 'NPkVill', 14: 'NWAmes', 15: 'NoRidge', 16: 'NridgHt', 17: 'OldTown', 18: 'SWISU', 19: 'Sawyer', 20: 'SawyerW', 21: 'Somerst', 22: 'StoneBr', 23: 'Timber', 24: 'Veenker'}
{0: '1.5Fin', 1: '1.5Unf', 2: '1Story', 3: '2.5Fin', 4: '2.5Unf', 5: '2Story', 6: 'SFoyer', 7: 'SLvl'}
{0: 'Flat', 1: 'Gable', 2: 'Gambrel', 3: 'Hip', 4: 'Mansard', 5: 'Shed'}
{0: 'AsbShng', 1: 'AsphShn', 2: 'BrkComm', 3: 'BrkFace', 4: 'CBlock', 5: 'CemntBd', 6: 'HdBoard', 7: 'ImStucc', 8: 'MetalSd', 9: 'Plywood', 10: 'Stone', 11: 'Stucco', 12: 'VinylSd', 13: 'Wd Sdng', 14: 'WdShing'}
{0: 'AsbShng', 1: 'AsphShn', 2: 'Brk Cmn', 3: 'BrkFace', 4: 'CBlock', 5: 'CmentBd', 6: 'HdBoard', 7: 'ImStucc', 8: 'MetalSd', 9: 'Other', 10: 'Plywood', 11: 'Stone', 12: 'Stucco', 13: 'VinylSd', 14: 'Wd Sdng', 15: 'Wd Shng'}
{0: 'BrkTil', 1: 'CBlock', 2: 

##Dimension reduction##


> Tìm số chiều cần giảm đến


In [ ]:
df_drop = df.drop(columns = 'Condition')

In [ ]:
pca = PCA().fit(df_drop)
nb_features = df_drop.shape[1] - 1

In [ ]:
var = 0.0
for k in range(1, nb_features + 1):
    pca = PCA(k)
    pca.fit(df_drop)

    newVar = pca.explained_variance_ratio_.sum() * 100
    print('   * k = %2d' %k, ': phương sai tích lũy ~ %.2f%%' %newVar,
          '--> tăng ~ %.2f%%' %(newVar - var))
    var = newVar

   * k =  1 : phương sai tích lũy ~ 99.60% --> tăng ~ 99.60%
   * k =  2 : phương sai tích lũy ~ 99.98% --> tăng ~ 0.38%
   * k =  3 : phương sai tích lũy ~ 99.99% --> tăng ~ 0.01%
   * k =  4 : phương sai tích lũy ~ 99.99% --> tăng ~ 0.01%
   * k =  5 : phương sai tích lũy ~ 100.00% --> tăng ~ 0.00%
   * k =  6 : phương sai tích lũy ~ 100.00% --> tăng ~ 0.00%
   * k =  7 : phương sai tích lũy ~ 100.00% --> tăng ~ 0.00%
   * k =  8 : phương sai tích lũy ~ 100.00% --> tăng ~ 0.00%
   * k =  9 : phương sai tích lũy ~ 100.00% --> tăng ~ 0.00%
   * k = 10 : phương sai tích lũy ~ 100.00% --> tăng ~ 0.00%
   * k = 11 : phương sai tích lũy ~ 100.00% --> tăng ~ 0.00%
   * k = 12 : phương sai tích lũy ~ 100.00% --> tăng ~ 0.00%
   * k = 13 : phương sai tích lũy ~ 100.00% --> tăng ~ 0.00%
   * k = 14 : phương sai tích lũy ~ 100.00% --> tăng ~ 0.00%
   * k = 15 : phương sai tích lũy ~ 100.00% --> tăng ~ 0.00%
   * k = 16 : phương sai tích lũy ~ 100.00% --> tăng ~ 0.00%
   * k = 17 : phương sai tí



**Chọn k = 5**



In [ ]:
data_norm = pd.DataFrame(StandardScaler().fit_transform(df_drop)) # tự động loại cột Class

## Áp dụng PCA
pca = PCA(n_components = 5)
pca.fit(data_norm)

PCA(n_components=5)

In [ ]:
B = pca.transform(data_norm)
df_pca = pd.DataFrame(B, columns = ['PC1', 'PC2', 'PC3', 'PC4', 'PC5'])
df_combined = pd.concat([df_pca, df.Condition], axis=1)
df_combined.head()

,PC1,PC2,PC3,PC4,PC5,Condition
0,3.354954,2.480342,-0.457747,-1.779032,0.535988,NOR
1,0.350617,-2.218229,-1.636814,-1.859659,-0.542116,NOR
2,4.101108,1.569868,-0.464637,-2.810664,-0.896718,NOR
3,-1.302402,-0.457290,1.478352,-0.736675,-1.569816,ABN
4,6.400934,0.439191,1.137185,-3.305372,-0.394818,NOR


# Machine learning

##Train model##

In [ ]:
X = df_combined.drop('Condition', axis = 1)
y = df_combined.Condition

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 1)

In [ ]:
## Phân tích giá trị của k theo chỉ số accuracy
list_k   = []
list_acc = []

for k in range(2, int(pow(X_train.shape[0], 1/2) / 2) + 2):
    list_k.append(k)

    neigh = KNeighborsClassifier(n_neighbors = k)
    neigh.fit(X_train, y_train)

    y_pred   = neigh.predict(X_test)
    acc      = accuracy_score(y_test, y_pred)
    list_acc.append(acc)
    print(f'k = {k:d} --> accuracy ~ {acc * 100:.1f}%')

k = 2 --> accuracy ~ 67.1%
k = 3 --> accuracy ~ 76.4%
k = 4 --> accuracy ~ 78.4%
k = 5 --> accuracy ~ 79.5%
k = 6 --> accuracy ~ 79.8%
k = 7 --> accuracy ~ 80.5%
k = 8 --> accuracy ~ 81.8%
k = 9 --> accuracy ~ 81.5%
k = 10 --> accuracy ~ 81.8%
k = 11 --> accuracy ~ 81.8%
k = 12 --> accuracy ~ 82.2%
k = 13 --> accuracy ~ 82.5%
k = 14 --> accuracy ~ 82.9%
k = 15 --> accuracy ~ 83.2%
k = 16 --> accuracy ~ 83.2%
k = 17 --> accuracy ~ 83.2%
k = 18 --> accuracy ~ 83.6%


In [ ]:
## Phân tích giá trị của k theo chỉ số f1
list_k   = []
list_f1 = []

for k in range(2, int(pow(X_train.shape[0], 1/2) / 2) + 2):
    list_k.append(k)

    neigh = KNeighborsClassifier(n_neighbors = k)
    neigh.fit(X_train, y_train)

    y_pred   = neigh.predict(X_test)
    f1      = f1_score(y_test, y_pred, average='micro')
    list_acc.append(f1)
    print(f'k = {k:d} --> f1 ~ {f1 * 100:.1f}%')

k = 2 --> f1 ~ 67.1%
k = 3 --> f1 ~ 76.4%
k = 4 --> f1 ~ 78.4%
k = 5 --> f1 ~ 79.5%
k = 6 --> f1 ~ 79.8%
k = 7 --> f1 ~ 80.5%
k = 8 --> f1 ~ 81.8%
k = 9 --> f1 ~ 81.5%
k = 10 --> f1 ~ 81.8%
k = 11 --> f1 ~ 81.8%
k = 12 --> f1 ~ 82.2%
k = 13 --> f1 ~ 82.5%
k = 14 --> f1 ~ 82.9%
k = 15 --> f1 ~ 83.2%
k = 16 --> f1 ~ 83.2%
k = 17 --> f1 ~ 83.2%
k = 18 --> f1 ~ 83.6%


**Ta thấy chọn k = 18 là phương án tốt nhất**

In [ ]:
k   = 18
knn = KNeighborsClassifier(n_neighbors = k)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=18)

In [ ]:
jlb.dump(knn, 'House_Condition_Predict.pkl')

['House_Condition_Predict.pkl']

In [ ]:
y_pred = knn.predict(X_test)

##Evaluating##

In [ ]:
def classification_eval(y_test, y_pred):
    accuracy  = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='micro')
    recall    = recall_score(y_test, y_pred, average='micro')
    f1        = f1_score(y_test, y_pred, average='micro')
    return accuracy, precision, recall, f1

In [ ]:
scores = classification_eval(y_test, y_pred)
y_proba = knn.predict_proba(X_test)
auc = roc_auc_score(y_test, y_proba, multi_class='ovr')

## Hiển thị giá trị cá chỉ số
print(f'Accuracy   = {(scores[0] * 100):.1f}%')
print(f'Precision = {(scores[1] * 100):.1f}%')
print(f'Recall    = {(scores[2] * 100):.1f}%')
print(f'F1        = {(scores[3] * 100):.1f}%')
print(f'AUC       = {(auc * 100):.1f}%')

Accuracy   = 83.6%
Precision = 83.6%
Recall    = 83.6%
F1        = 83.6%
AUC       = 62.7%


**Ta thấy được các chỉ số accuracy, precision, recall, f1 đều đạt mức khá cao cho thấy mô hình có độ chính xác tốt.**

**Với chỉ số AUC do đường cong ROC thường hiệu quả hơn với phân lớp nhị phân do vậy trong trường hợp này chỉ số này thấp hơn nhiều so với các chỉ số còn lại**

In [ ]:
cv_scores = cross_val_score(knn, X, y, scoring = 'accuracy', cv = 10)  # Áp dụng 10-fold cross-validation
# In ra độ chính xác trung bình và độ lệch chuẩn của các lần cross-validation
print(f"Accuracy dao động trong khoảng: {round(np.min(cv_scores) * 100,2)}% - {round(np.max(cv_scores) * 100,2)}%")
print(f"Accuracy trung bình: {round(np.mean(cv_scores)*100,2)}%")
print(f"Độ lệch chuẩn của Accuracy: {round(np.std(cv_scores),2)}")

Accuracy dao động trong khoảng: 80.82% - 84.93%
Accuracy trung bình: 82.53%
Độ lệch chuẩn của Accuracy: 0.01


**Sử dụng phương pháp k-fold cross validation với k = 10 ta thấy được mô hình có tính chính xác tốt, duy trì điểm chỉ số accuracy không biến thiên quá mạnh giữa các fold**
